In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from copy import deepcopy
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import sys 
import time

BATCH_SIZE = 10

print(tf.__version__)

2.7.0


In [ ]:
path_training ="/content/drive/MyDrive/IC-Colab-CSV files/01_Table3Plus7_LessLines.csv"
path_test ="/content/drive/MyDrive/IC-Colab-CSV files/02_Table3Plus7.csv"
df = pd.read_csv(path_training)
df_test = pd.read_csv(path_test)
df


,ServerPort,59,60,82,83,95,96,97,113,137,target
0,80,1,1,140,319,319,140,1,3,0,WWW
1,80,1,1,141,319,319,141,1,3,0,WWW
2,80,1,1,141,322,322,141,1,3,0,WWW
3,80,1,1,142,319,319,142,1,3,0,WWW
4,80,1,1,141,325,325,141,1,3,0,WWW
...,...,...,...,...,...,...,...,...,...,...,...
23796,80,1,1,388,450,450,388,1,3,0,WWW
23797,80,1,1,536,447,447,783,1,3,0,WWW
23798,80,1,1,629,485,485,629,1,3,0,WWW
23799,80,1,1,390,450,450,390,1,3,0,WWW


In [ ]:
#df.dtypes

In [ ]:
target = df.pop('target')
target_test = df_test.pop('target')
#Serverport = df.pop('ServerPort')
df

,ServerPort,59,60,82,83,95,96,97,113,137
0,80,1,1,140,319,319,140,1,3,0
1,80,1,1,141,319,319,141,1,3,0
2,80,1,1,141,322,322,141,1,3,0
3,80,1,1,142,319,319,142,1,3,0
4,80,1,1,141,325,325,141,1,3,0
...,...,...,...,...,...,...,...,...,...,...
23796,80,1,1,388,450,450,388,1,3,0
23797,80,1,1,536,447,447,783,1,3,0
23798,80,1,1,629,485,485,629,1,3,0
23799,80,1,1,390,450,450,390,1,3,0


In [ ]:
classes = ['WWW','MAIL','FTP-CONTROL','FTP-PASV','ATTACK','P2P','DATABASE','FTP-DATA','MULTIMEDIA','SERVICES','INTERACTIVE','GAMES']

#classes = list(target.unique())
#classes



In [ ]:
#Converts the above list of classes to an array of index
target = target.apply(lambda x: classes.index(x))
target.unique()
target_test = target_test.apply(lambda x: classes.index(x))

In [ ]:
numeric_feature_names = ['ServerPort','59','60','82','83','95','96','97','113','137']
numeric_features = df[numeric_feature_names] # Data for training
numeric_features_test = df_test[numeric_feature_names] # Data for testing
numeric_features.head()

,ServerPort,59,60,82,83,95,96,97,113,137
0,80,1,1,140,319,319,140,1,3,0
1,80,1,1,141,319,319,141,1,3,0
2,80,1,1,141,322,322,141,1,3,0
3,80,1,1,142,319,319,142,1,3,0
4,80,1,1,141,325,325,141,1,3,0


In [ ]:
#converting pandas dataframe to tensorflow object
numeric_features = tf.convert_to_tensor(numeric_features)
numeric_features_test = tf.convert_to_tensor(numeric_features_test)
numeric_features

<tf.Tensor: shape=(23801, 10), dtype=int64, numpy=
array([[80,  1,  1, ...,  1,  3,  0],
       [80,  1,  1, ...,  1,  3,  0],
       [80,  1,  1, ...,  1,  3,  0],
       ...,
       [80,  1,  1, ...,  1,  3,  0],
       [80,  1,  1, ...,  1,  3,  0],
       [80,  1,  1, ...,  1,  3,  0]])>

In [ ]:
# Normalize the data
normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(numeric_features)
#normalizer.adapt(numeric_features_test)

Ideia: Fazer a normalização entre 0 e 1 conforme o Andrews, aqui atualmente ele está centrado em 0 com desvio padrão em 1

In [ ]:
def get_basic_model():
  model = tf.keras.Sequential([
    normalizer,
    tf.keras.layers.Dense(10, activation='tanh'), 
    tf.keras.layers.Dense(12)
  ])

  model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
  return model

Requisição Weverton 
Como recuperar os pesos após os treinos? Descobrir o método

In [ ]:
model = get_basic_model()
model.fit(numeric_features, target, epochs=10, batch_size=BATCH_SIZE)

Epoch 1/10
2381/2381 [==============================] - 4s 2ms/step - loss: 0.4738 - accuracy: 0.8999
Epoch 2/10
2381/2381 [==============================] - 4s 2ms/step - loss: 0.1483 - accuracy: 0.9595
Epoch 3/10
2381/2381 [==============================] - 4s 2ms/step - loss: 0.1090 - accuracy: 0.9669
Epoch 4/10
2381/2381 [==============================] - 4s 2ms/step - loss: 0.0930 - accuracy: 0.9747
Epoch 5/10
2381/2381 [==============================] - 4s 2ms/step - loss: 0.0843 - accuracy: 0.9771
Epoch 6/10
2381/2381 [==============================] - 4s 2ms/step - loss: 0.0785 - accuracy: 0.9805
Epoch 7/10
2381/2381 [==============================] - 4s 2ms/step - loss: 0.0741 - accuracy: 0.9818
Epoch 8/10
2381/2381 [==============================] - 4s 2ms/step - loss: 0.0701 - accuracy: 0.9826
Epoch 9/10
2381/2381 [==============================] - 4s 2ms/step - loss: 0.0666 - accuracy: 0.9834
Epoch 10/10
2381/2381 [==============================] - 4s 2ms/step - loss: 0.063

In [ ]:
test_loss, test_acc = model.evaluate(numeric_features_test,  target_test, verbose=2, batch_size=BATCH_SIZE) 

print('\nTest accuracy:', test_acc)



2381/2381 - 2s - loss: 0.1428 - accuracy: 0.9620 - 2s/epoch - 1ms/step

Test accuracy: 0.9619764089584351
